# Tradução de Inglês para Português utilizando um Modelo Transformer Sequence-to-Sequence

## Introdução
Neste exemplo, vamos construir um modelo Transformer de sequência para sequência (sequence-to-sequence ), que será treinado em uma tarefa de tradução automática do inglês para o português.

Você aprenderá:

- Vetorizar texto usando a camada TextVectorization do Keras.
- Implementar uma camada TransformerEncoder, uma camada TransformerDecoder e uma camada PositionalEmbedding.
- Preparar os dados para o treinamento de um modelo de *sequence-to-sequence*.
- Usar o modelo treinado para gerar traduções de frases nunca vistas anteriormente.

O código original foi adaptado do livro Deep Learning with Python <(https://www.manning.com/books/deep-learning-with-python-second-edition)> e dos exemplos do Keras <https://keras.io/examples/nlp/neural_machine_translation_with_transformer/>


#**Importando as bibliotecas**

In [2]:
import random
import numpy as np
import tensorflow as tf
import keras
from keras import layers, ops

**Analisando os dados**

Cada linha contém uma frase em inglês e sua respectiva tradução em português.
A frase em inglês é a sequência de origem e a frase em inglês é a sequência de destino.
Adicionamos o token "[start]" no início e o token "[end]" no final da frase em português.

In [3]:
# Abre o arquivo chamado 'port4.txt' no modo leitura
# O arquivo é atribuído à variável f
#
with open('port4.txt') as f:
    lines = f.read().split("\n")[:-1] # Lê todas as linhas do arquivo e as divide em uma lista, removendo a última linha vazia
text_pairs = [] # Inicializa uma lista vazia para armazenar os pares de texto inglês -> português


#**Formatando os pares de frases:**

In [4]:
# Itera sobre cada linha da lista 'lines', onde cada linha contém uma frase em inglês
# e sua tradução em português, separadas por tabulação (\t)
for line in lines:

    # Divide a linha usando '\t' como delimitador e atribui as partes às variáveis 'eng' e 'por'.
    # 'eng' = frase em inglês (sequência de entrada)
    # 'por' = frase em português (sequência de saída, antes de adicionar os tokens)
    eng, por = line.split("\t")

    # Adiciona os tokens especiais "[start]" no início e "[end]" no final da frase em português.
    # Isso ajuda o modelo seq2seq a reconhecer o início e o fim da sequência de saída durante a tradução.
    por = "[start] " + por + " [end]"

    # Adiciona o par processado (frase em inglês, frase em português com tokens) à lista 'text_pairs'
    text_pairs.append((eng, por))

# Imprime o primeiro item da lista 'text_pairs' (o índice começa em 0), ou seja,
# o primeiro par de tradução processado
print(text_pairs[0])

('Go.', '[start] Vai. [end]')


# **Criando os conjuntos de treinamento, validação e teste.**

In [5]:
# Embaralha a lista de pares de texto (text_pairs)
# Isso é importante antes de dividir os dados em conjuntos de treino, validação e teste
random.shuffle(text_pairs)

# Calcula o número de amostras para o conjunto de validação (15% do total)
num_val_samples = int(0.15 * len(text_pairs))

# Calcula o número de amostras para o conjunto de treinamento
# O restante será dividido entre validação e teste (70% para treino, 15% para validação, 15% para teste)
num_train_samples = len(text_pairs) - 2 * num_val_samples

# Define o conjunto de treinamento: primeiras 'num_train_samples' amostras
train_pairs = text_pairs[:num_train_samples]

# Define o conjunto de validação: próximas 'num_val_samples' amostras após o treinamento
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]

# Define o conjunto de teste: amostras restantes após o treinamento e validação
test_pairs = text_pairs[num_train_samples + num_val_samples :]


# Imprime informações sobre o tamanho total de pares
print(f"{len(text_pairs)} total de pares do conjunto de amostras")

# Imprime o número de pares de frases usadas para treinamento
print(f"{len(train_pairs)} conjunto de treinamento")

# Imprime o número de pares de frases usadas para validação
print(f"{len(val_pairs)} conjunto de validação")

# Imprime o número de pares de frases usadas para teste
print(f"{len(test_pairs)} conjunto de teste")

196350 total de pares do conjunto de amostras
137446 conjunto de treinamento
29452 conjunto de validação
29452 conjunto de teste


 **Vetorizando os dados de texto**

Usaremos duas instâncias da camada TextVectorization para vetorizar os dados de texto (uma para o inglês e uma para o português), ou seja, transformar as strings originais em sequências de inteiros, onde cada inteiro representa o índice de uma palavra no vocabulário.

In [6]:
# Define o tamanho máximo do vocabulário (número máximo de palavras a serem consideradas)
vocab_size = 15000

# Define o comprimento máximo das sequências (cada frase será truncada ou preenchida até esse valor)
sequence_length = 20

# Define o número de amostras por lote durante o treinamento do modelo
batch_size = 64

# Cria uma camada TextVectorization para o inglês
eng_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,          # Máximo de tokens no vocabulário
    output_mode="int",              # Saída como números inteiros (índices de palavras)
    output_sequence_length=sequence_length,  # Comprimento fixo das sequências
)

# Cria uma camada TextVectorization para o português
por_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,          # Mesmo tamanho de vocabulário
    output_mode="int",              # Saída como números inteiros
    output_sequence_length=sequence_length + 1,  # Um pouco mais longa (para incluir [start] e [end])
    #standardize=custom_standardization,
)

# Extrai apenas as frases em inglês dos pares de treinamento
train_eng_texts = [pair[0] for pair in train_pairs]

# Extrai apenas as frases em português dos pares de treinamento
train_por_texts = [pair[1] for pair in train_pairs]

# Ajusta (adapta) o vetorizador de inglês ao conjunto de dados de treino
eng_vectorization.adapt(train_eng_texts)

# Ajusta (adapta) o vetorizador de português ao conjunto de dados de treino
por_vectorization.adapt(train_por_texts)

#**Formatando os conjuntos de dados**

Em cada passo do treinamento, o modelo tentará prever as palavras N+1 (e as posteriores) da sequência alvo usando:

a frase de entrada, e as palavras de destino de 0 até N.
Dessa forma, o conjunto de dados de treinamento retornará uma tupla (inputs, targets), onde:

* *inputs* é um dicionário com as chaves *encoder_inputs* e *decoder_inputs*.

* *encoder_inputs* é a frase fonte vetorizada.

* *decoder_inputs* é a frase alvo "até o momento", ou seja, as palavras de 0 até N usadas para prever a palavra N+1 (e as posteriores) na frase alvo.

* *target* é a frase alvo deslocada em um passo:
ela fornece as próximas palavras na frase alvo — aquelas que o modelo tentará prever.

In [7]:
def format_dataset(eng, por):
    # Vetoriza as frases em inglês usando a camada TextVectorization pré-adaptada
    eng = eng_vectorization(eng)

    # Vetoriza as frases em português também
    por = por_vectorization(por)

    # Retorna uma tupla com dois elementos:
    # 1. Um dicionário contendo os inputs para o encoder e o decoder
    #    - "encoder_inputs": sequências em inglês vetorizadas
    #    - "decoder_inputs": sequências em português *até o penúltimo token*, usadas como entrada do decoder
    # 2. O alvo ("target") para o treinamento: a mesma sequência em português, mas iniciando
    # a partir do segundo token. Isso permite prever a próxima palavra a partir da anterior
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": por[:, :-1],
        },
        por[:, 1:],
    )

def make_dataset(pairs):
    # Separa os pares bilíngues em duas listas: textos em inglês e textos em português
    eng_texts, por_texts = zip(*pairs)

    # Converte as tuplas resultantes em listas (para compatibilidade com o TensorFlow)
    eng_texts = list(eng_texts)
    por_texts = list(por_texts)

    # Cria um dataset do TensorFlow a partir dos pares de texto
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, por_texts))

    # Agrupa os exemplos em lotes (batches) de tamanho `batch_size`
    dataset = dataset.batch(batch_size)

    # Aplica a função `format_dataset` a cada lote do dataset
    # Isso transforma texto em vetores e prepara os pares de entrada/saída para o modelo
    dataset = dataset.map(format_dataset)

    # Retorna o dataset com cache (para evitar recalcular), embaralhado e com prefetch (pré-carregamento)
    return dataset.cache().shuffle(2048).prefetch(16)


# Cria o dataset de treinamento a partir dos pares de treino
train_ds = make_dataset(train_pairs)

# Cria o dataset de validação a partir dos pares de validação
val_ds = make_dataset(val_pairs)

# Conferindo as sequencias: temos lotes de 64 pares e todas as sequências têm 20 passos de comprimento.
for inputs, targets in train_ds.take(1):
  print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
  print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
  print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


# **Contruindo o Modelo**

Nosso modelo Transformer *sequence to sequence* é composto por um *Encoder* e um *Decoder* conectados em sequência. Para que o modelo reconheça a ordem das palavras, também utilizamos uma camada de *PositionalEmbedding* (Inserção de Posição).

A sequência de origem será passada para o *Encoder*, que produzirá uma nova representação dela. Essa nova representação será então enviada ao *Decoder*, junto com a sequência de destino até o momento (palavras de destino de 0 até N). O *Decoder* terá como objetivo prever as próximas palavras na sequência de destino (N+1 e seguintes).

Um detalhe fundamental que torna isso possível é a máscara causal (veja o método *get_causal_attention_mask()* no *Decoder*). O *Decoder* recebe toda a sequência de uma vez, e por isso precisamos garantir que ele utilize apenas informações dos tokens de destino de 0 até N ao prever o token N+1 (caso contrário, poderia usar informações do futuro, o que resultaria em um modelo que não pode ser usado durante a inferência).

In [8]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        # Inicializa a camada base (Layer)
        super().__init__(**kwargs)

        # Parâmetros importantes do encoder:
        self.embed_dim = embed_dim   # Dimensão da embedding
        self.dense_dim = dense_dim   # Dimensão da camada densa interna
        self.num_heads = num_heads   # Número de heads na atenção multi-head

        # Camada de atenção multi-head
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )

        # Rede feed-forward (projecção densa) com duas camadas
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

        # Normalização das camadas (duas instâncias: uma antes e outra depois da rede densa)
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

        # Indica que esta camada suporta máscara (para lidar com sequências de diferentes comprimentos)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        # Se houver máscara, converte-a para formato adequado
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        # Atenção multi-head: Q=V=K=inputs
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )

        # Residual + LayerNorm 1
        proj_input = self.layernorm_1(inputs + attention_output)

        # Passa pela rede densa
        proj_output = self.dense_proj(proj_input)

        # Residual + LayerNorm 2
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        # Método usado para serializar a configuração da camada
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)

        # Camada de embeddings dos tokens (vocabulário)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )

        # Camada de embeddings das posições (posições no tempo/sequência)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )

        # Guarda os parâmetros como atributos
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        # Pega o comprimento da sequência
        length = ops.shape(inputs)[-1]

        # Gera as posições (0 até length - 1)
        positions = ops.arange(0, length, 1)

        # Aplica a embedding dos tokens
        embedded_tokens = self.token_embeddings(inputs)

        # Aplica a embedding das posições
        embedded_positions = self.position_embeddings(positions)

        # Soma os dois embeddings (tokens + posições)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Define a máscara: ignora tokens com valor zero (padrão de preenchimento)
        return ops.not_equal(inputs, 0)

    def get_config(self):
        # Serializa a configuração da camada
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        # Parâmetros principais do decoder
        self.embed_dim = embed_dim     # Dimensão da embedding
        self.latent_dim = latent_dim # Dimensão latente da rede densa
        self.num_heads = num_heads   # Número de heads na atenção multi-head

        # Primeira atenção: autoatenção dentro do próprio decoder
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )

        # Segunda atenção: entre decoder e saída do encoder
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )

        # Rede densa do decoder
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

        # Três normalizações de camada para residual learning
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        # Suporte à máscara (importante para inferência autoregressiva)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        # Separa entrada do decoder e saída do encoder
        inputs, encoder_outputs = inputs

        # Gera a máscara causal (evita olhar para o futuro)
        causal_mask = self.get_causal_attention_mask(inputs)

        # Extrai máscaras de padding, se existirem
        if mask is None:
            inputs_padding_mask, encoder_outputs_padding_mask = None, None
        else:
            inputs_padding_mask, encoder_outputs_padding_mask = mask

        # Autoatenção: decoder consigo mesmo
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask,
            query_mask=inputs_padding_mask,
        )

        # Residual + LayerNorm 1
        out_1 = self.layernorm_1(inputs + attention_output_1)

        # Atenção ao encoder (cross-attention)
        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=inputs_padding_mask,
            key_mask=encoder_outputs_padding_mask,
        )

        # Residual + LayerNorm 2
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        # Rede densa final
        proj_output = self.dense_proj(out_2)

        # Residual + LayerNorm 3
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        # Cria a máscara causal para evitar leak de informações futuras
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]

        # Cria matriz triangular inferior (cada posição só vê o passado)
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")  # [length, length]

        # Expande a máscara para todos os elementos do batch
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        # Serializa a configuração da camada
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


# **Modelo**

In [12]:
# Define a dimensão dos embeddings (tamanho do vetor usado para representar palavras)
embed_dim = 256

# Dimensão latente usada nas camadas densas internas das camadas Transformer
latent_dim = 2048

# Número de heads na atenção multi-head - permite ao modelo aprender diferentes relações simultaneamente
num_heads = 8

# Cria a entrada para o encoder: sequência de tokens inteiros (palavras indexadas)
encoder_inputs = keras.Input(
    shape=(None,),  # Sequências de comprimento variável
    dtype="int64",   # Tokens são números inteiros (índices no vocabulário)
    name="encoder_inputs" # Nome da camada de entrada
)

# Aplica a embedding posicional às entradas do encoder:
# Combina a embedding das palavras com a informação de posição
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Passa a entrada pelo encoder do Transformer:
# Transforma a sequência fonte em uma representação contextualizada
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

# Define o modelo do encoder: mapeia as entradas para a saída do encoder
encoder = keras.Model(encoder_inputs, encoder_outputs)

# Cria a entrada para o decoder: sequência de tokens (frase alvo até o momento)
decoder_inputs = keras.Input(
    shape=(None,),
    dtype="int64",
    name="decoder_inputs"
)

# Entrada adicional para o decoder: saída do encoder (representação da frase fonte)
encoded_seq_inputs = keras.Input(
    shape=(None, embed_dim),
    name="decoder_state_inputs"
)

## Aplica a embedding posicional à sequência de entrada do decoder
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Passa a entrada e o estado do encoder para o decoder do Transformer:
# O decoder usa a saída do encoder para gerar a próxima palavra da sequência
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])

# Adiciona dropout para regularização (evitar overfitting)
x = layers.Dropout(0.5)(x)

# Camada final: projeta os vetores do decoder para o tamanho do vocabulário,
# produzindo probabilidades para cada palavra possível (usando softmax)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Define o modelo do decoder como um submodelo:
# Recebe a entrada do decoder e o estado vindo do encoder, e gera a saída
decoder = keras.Model(
    [decoder_inputs, encoded_seq_inputs],
    decoder_outputs
)

# Monta o modelo completo do Transformer:
# Conecta as partes do encoder e do decoder
transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

# **Treinando o Modelo**

Vamos usar a *acurácia* como métrica para monitorar o progresso do treinamento nos dados de validação.
A tradução automática muitas vezes utiliza também a métrica BLEU (*Bilingual Evaluation Understudy*).

No exemplo, vamos treinar por apenas 1 época, mas para que o modelo convergir, é necessário treiná-lo por pelo menos 30 épocas.

In [10]:
epochs = 1  # é necessário treiná-lo por pelo menos 30 épocas.

transformer.summary()
transformer.compile(
    "rmsprop",
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["accuracy"],
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)



Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 256) │  3,155,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, None)      │          0 │ decoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decoder │ (None, None, 256) │  5,259,520 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
│                     │                   │            │ not_equal_1[0][0… │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, None, 256) │          0 │ transformer_deco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │  3,855,000 │ dropout_3[0][0]   │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

KeyboardInterrupt: 

## **Traduzindo as frase**

Finalmente, vamos demonstrar como traduzir frases em inglês completamente novas, que não foram usadas no treinamento.

Basta fornecer ao modelo a frase em inglês vetorizada e o token alvo "[start]", em seguida geramos repetidamente o próximo token, até que alcancemos o token "[end]".

In [ ]:
# Obtém o vocabulário do vetorizador de português
por_vocab = por_vectorization.get_vocabulary()

# Cria um dicionário que mapeia índices para palavras no vocabulário
# Isso permite converter índices previstos pelo modelo de volta para texto
por_index_lookup = dict(zip(range(len(por_vocab)), por_vocab))

# Define o comprimento máximo da frase traduzida gerada durante a inferência
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence]) # Vetoriza a frase de entrada (em inglês)
    decoded_sentence = "[start]"  # Inicia a frase de saída com o token "[start]", indicando o início da tradução
    # Loop para gerar a tradução palavra por palavra, até o limite definido
    for i in range(max_decoded_sentence_length):
        # Vetoriza a frase em português atual (até agora), excluindo o último token
        tokenized_target_sentence = por_vectorization([decoded_sentence])[:, :-1]
        # Passa as entradas pelo modelo:
        # - tokenized_input_sentence: entrada do encoder (frase em inglês)
        # - tokenized_target_sentence: entrada do decoder (até agora)
        predictions = transformer(
            {
                "encoder_inputs": tokenized_input_sentence,
                "decoder_inputs": tokenized_target_sentence,
            }
        )

        # Encontra o índice da palavra mais provável na posição atual
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)

        # Converte o índice para a palavra real usando o dicionário de lookup
        sampled_token = por_index_lookup[sampled_token_index]

        # Adiciona a nova palavra à frase traduzida
        decoded_sentence += " " + sampled_token

        # Se o token final "[end]" for alcançado, interrompe a geração
        if sampled_token == "[end]":
            break
    # Retorna a frase traduzida completa
    return decoded_sentence

# Extrai apenas as frases em inglês dos pares de teste
test_eng_texts = [pair[0] for pair in test_pairs]

# Traduz e imprime 30 frases aleatórias do conjunto de teste
for _ in range(30):
    input_sentence = random.choice(test_eng_texts) # Escolhe uma frase aleatória do conjunto de teste
    translated = decode_sequence(input_sentence)   # Gera a tradução usando o modelo treinado
    print(f'{input_sentence} -> {translated}')     # Imprime a entrada e a tradução gerada